# FPL Data Generator

This notebook scrapes played matches, then generates the required CSV files for the accompanying Tableau dashboards.

In [1]:
#Standard data manipulations
import pandas as pd
import numpy as np

from datetime import date
import sqlite3
import pickle

#Import custom functions
from PythonFunctions.apicallers import *
from PythonFunctions.sqlfunctions import *
from PythonFunctions.matchscrapers import *

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

#Connect to the SQL database
conn = sqlite3.connect('Data/20_21fpl.db')
#Instantiate a cursor
c = conn.cursor()

Work out which matches we ought to have data for, which aren't present in the database.

In [2]:
to_run = suggested_match_ids()
print(to_run)

[58919, 58920, 58921, 58922, 58923, 58925]


Update data for these matches.

In [3]:
# CoreDataUpdater(to_run, c, conn)

Sweep up matches that the updater might have missed.

In [4]:
# MatchSweeper(c, conn)

# Expected Goals

In [5]:
#Bring in XG functions now (note this is brought in after the new goal data is known)
from PythonFunctions.xg import *

In [6]:
#Generate a player detail dataframe, with the XG stats 
df_pm = pd.merge(df_pm_generator(), api_stat_generator(), on=['MatchID','Player'])
df_tm = df_tm_generator()